In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd

In [2]:
with open('reviews.txt', 'r') as f:
    reviews= f.read()

In [3]:
df = pd.read_table('reviews.txt', names=["message"])

In [4]:
df1 = pd.read_table('labels.txt',names=['labels'])

In [5]:
data = df_all_rows = pd.concat([df, df1],axis = 1)

In [6]:
data

,message,labels
0,bromwell high is a cartoon comedy . it ran at ...,positive
1,story of a man who has unnatural feelings for ...,negative
2,homelessness or houselessness as george carli...,positive
3,airport starts as a brand new luxury pla...,negative
4,brilliant over acting by lesley ann warren . ...,positive
...,...,...
24995,i saw descent last night at the stockholm fi...,negative
24996,a christmas together actually came before my t...,positive
24997,some films that you pick up for a pound turn o...,negative
24998,working class romantic drama from director ma...,positive


In [8]:
data = data[:1000]

In [9]:
data

,message,labels
0,bromwell high is a cartoon comedy . it ran at ...,positive
1,story of a man who has unnatural feelings for ...,negative
2,homelessness or houselessness as george carli...,positive
3,airport starts as a brand new luxury pla...,negative
4,brilliant over acting by lesley ann warren . ...,positive
...,...,...
995,oh dear lord how on earth was any part of thi...,negative
996,i had never heard of this film before a couple...,positive
997,this is not a film you can really analyse sepa...,negative
998,many of the criticisms on this thread seem to ...,positive


In [10]:
#Data cleaning and preprocessing
import re
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Delll\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [12]:
corpus = []
for i in range(0,len(data)):
    review = re.sub('[^a-zA-Z0-9]',' ',data['message'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
review

In [ ]:
corpus

In [127]:
# Creating the Bag of Words model

In [128]:
from sklearn.feature_extraction.text import CountVectorizer

In [129]:
cv = CountVectorizer(max_features=2500)

In [130]:
X = cv.fit_transform(corpus).toarray()

In [131]:
X.shape

(5000, 2500)

In [132]:
 data['labels']=[1 if i == 'positive' else 0 for i in data['labels'] ]

C:\Users\Delll\anaconda3\envs\nlp_emmbeding\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [133]:
data

,message,labels
0,bromwell high is a cartoon comedy . it ran at ...,1
1,story of a man who has unnatural feelings for ...,0
2,homelessness or houselessness as george carli...,1
3,airport starts as a brand new luxury pla...,0
4,brilliant over acting by lesley ann warren . ...,1
...,...,...
4995,the only real highlight in the movie is the de...,0
4996,old jane s mannered tale seems very popular t...,1
4997,you know i can take the blood and the sex b...,0
4998,this schiffer guy is a real genius the movie ...,1


In [134]:
X.shape

(5000, 2500)

In [13]:
##tensorflow >2.0
from tensorflow.keras.preprocessing.text import one_hot

In [14]:
### Vocabulary size
voc_size=10000

In [15]:
onehot_repr =[one_hot(word,voc_size) for word in corpus]

In [16]:
onehot_repr

[[8548,
  7824,
  5156,
  6008,
  2656,
  5076,
  1739,
  6178,
  2386,
  5085,
  7653,
  3190,
  3402,
  8142,
  9480,
  8548,
  7824,
  3467,
  863,
  4885,
  8880,
  5085,
  5248,
  3349,
  4629,
  5966,
  3199,
  6985,
  3386,
  3373,
  5085,
  658,
  1956,
  9992,
  573,
  1901,
  6178,
  7430,
  3199,
  2008,
  6036,
  3199,
  3172,
  6079,
  4903,
  6178,
  3238,
  5789,
  7824,
  8029,
  341,
  9825,
  6689,
  3339,
  5085,
  3199,
  7574,
  8548,
  7824,
  7655,
  3420,
  5107,
  5365,
  5045,
  8548,
  7824,
  7250,
  7250,
  8912],
 [3180,
  8599,
  6397,
  8501,
  577,
  571,
  661,
  7857,
  3839,
  3735,
  3550,
  6008,
  8910,
  5758,
  2475,
  1142,
  4978,
  9960,
  8861,
  2862,
  7557,
  8540,
  5024,
  1806,
  3550,
  9992,
  5076,
  8528,
  7543,
  882,
  102,
  1779,
  9451,
  7208,
  1142,
  6737,
  1611,
  8264,
  102,
  8845,
  8467,
  8082,
  2197,
  8917,
  8202,
  2899,
  6576,
  3261,
  5045,
  5055,
  3983,
  2210,
  2736,
  1621,
  502,
  2210,
  4330,
  

In [18]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
import numpy as np

In [19]:
sent_length = 500
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)


[[   0    0    0 ... 7250 7250 8912]
 [   0    0    0 ... 1226 7604 3119]
 [   0    0    0 ... 4719 7325 3226]
 ...
 [   0    0    0 ... 1320 2647 1369]
 [   0    0    0 ... 4514 3954 2537]
 [   0    0    0 ... 6198 7570 2134]]


In [20]:
embedded_docs.shape

(1000, 500)

In [45]:
X = np.reshape(embedded_docs,(1000,500,1))

In [46]:
X.shape

(1000, 500, 1)

In [25]:
data['labels'].shape

(1000,)

In [26]:
y = data['labels']

In [27]:
y

0      positive
1      negative
2      positive
3      negative
4      positive
         ...   
995    negative
996    positive
997    negative
998    positive
999    negative
Name: labels, Length: 1000, dtype: object

In [28]:
y = np.reshape(y.to_numpy(), ( -1,1))

In [50]:
y.shape

(1000, 1)

In [53]:
y=[1 if i == 'positive' else 0 for i in y ]

In [54]:
split_frac = 0.8
split_idx = int(len(X) * split_frac)

train_x, val_x = X[:split_idx], X[split_idx:]
train_y, val_y = y[:split_idx], y[:split_idx]

test_idx = int(len(val_x) * 0.5)
val_x, test_x = val_x[:test_idx], val_x[test_idx:]
val_y, test_y = val_y[:test_idx], val_y[test_idx:]

print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape), 
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))

			Feature Shapes:
Train set: 		(800, 500, 1) 
Validation set: 	(100, 500, 1) 
Test set: 		(100, 500, 1)


In [31]:
import numpy as np

In [32]:
from sklearn.metrics import accuracy_score

In [33]:
from keras.datasets import reuters

Using TensorFlow backend.


In [34]:
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Activation
from keras import optimizers
from keras.wrappers.scikit_learn import KerasClassifier


In [182]:
def vanilla_rnn():
    model = Sequential()
    model.add(SimpleRNN(100, input_shape = (500,1), return_sequences = False))
    model.add(Dense(46))
    model.add(Activation('sigmoid'))
    
    adam = optimizers.Adam(lr = 0.001)
    model.compile(loss = 'sparse_categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    return model

In [187]:
model = KerasClassifier(build_fn = vanilla_rnn, epochs = 10, batch_size = 10, verbose = 1)

In [188]:
model.fit(train_x, train_y)

Epoch 1/10
4000/4000 [==============================] - 75s 19ms/step - loss: 0.9084 - accuracy: 0.4882
Epoch 2/10
4000/4000 [==============================] - 79s 20ms/step - loss: 0.6966 - accuracy: 0.5000
Epoch 3/10
4000/4000 [==============================] - 82s 21ms/step - loss: 0.6946 - accuracy: 0.5000
Epoch 4/10
4000/4000 [==============================] - 86s 21ms/step - loss: 0.6940 - accuracy: 0.4960
Epoch 5/10
4000/4000 [==============================] - 75s 19ms/step - loss: 0.6937 - accuracy: 0.5000
Epoch 6/10
4000/4000 [==============================] - 89s 22ms/step - loss: 0.6935 - accuracy: 0.5000
Epoch 7/10
4000/4000 [==============================] - 93s 23ms/step - loss: 0.6934 - accuracy: 0.5000
Epoch 8/10
4000/4000 [==============================] - 77s 19ms/step - loss: 0.6934 - accuracy: 0.5002
Epoch 9/10
4000/4000 [==============================] - 82s 21ms/step - loss: 0.6933 - accuracy: 0.5000
Epoch 10/10
4000/4000 [==============================] - 84s 21m

In [67]:
X.shape

(1000, 250, 1)

In [68]:
y.shape

(1000, 1)

In [90]:
model.predict(np.reshape(test_x[10],(1,200,1)))

1/1 [==============================] - 0s 51ms/step


array([0], dtype=int64)

In [91]:
y[10]

array([1], dtype=int64)

In [35]:
from keras.datasets import imdb
from keras.layers import GRU, LSTM, CuDNNGRU, CuDNNLSTM, Activation
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential

In [47]:
def lstm_model():
  model = Sequential()
  model.add(LSTM(50,input_shape=(500,1),return_sequences=True))
  model.add(LSTM(1,return_sequences=False))
  model.add(Activation('sigmoid'))

  model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
  return model

In [48]:
model = lstm_model()

In [55]:
model.fit(train_x,train_y, batch_size=100, epochs=10, verbose=1)

Epoch 1/10
800/800 [==============================] - 11s 13ms/step - loss: 0.7059 - accuracy: 0.5000
Epoch 2/10
800/800 [==============================] - 7s 9ms/step - loss: 0.7019 - accuracy: 0.5000
Epoch 3/10
800/800 [==============================] - 8s 10ms/step - loss: 0.6979 - accuracy: 0.5000
Epoch 4/10
800/800 [==============================] - 8s 11ms/step - loss: 0.6935 - accuracy: 0.4863
Epoch 5/10
800/800 [==============================] - 8s 11ms/step - loss: 0.6931 - accuracy: 0.5038
Epoch 6/10
800/800 [==============================] - 9s 11ms/step - loss: 0.6931 - accuracy: 0.5025
Epoch 7/10
800/800 [==============================] - 9s 11ms/step - loss: 0.6931 - accuracy: 0.5025
Epoch 8/10
800/800 [==============================] - 9s 12ms/step - loss: 0.6931 - accuracy: 0.5025
Epoch 9/10
800/800 [==============================] - 9s 11ms/step - loss: 0.6931 - accuracy: 0.5025
Epoch 10/10
800/800 [==============================] - 9s 11ms/step - loss: 0.6931 - accura